## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import google_api_keys as g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Vao,NC,-22.6667,167.4833,69.62,62,99,11.74,overcast clouds
1,1,Busselton,AU,-33.6500,115.3333,58.51,71,100,15.70,overcast clouds
2,2,Hilo,US,19.7297,-155.0900,88.29,72,40,11.50,scattered clouds
3,3,Betanzos,ES,43.2804,-8.2147,66.85,86,40,2.30,scattered clouds
4,4,Uglovskoye,RU,51.3557,80.1942,64.60,69,28,6.33,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
5,5,Rikitea,PF,-23.1203,-134.9692,73.47,72,76,21.59,broken clouds
15,15,Tura,IN,25.5198,90.2201,73.72,92,46,2.98,scattered clouds
21,21,Fortuna,US,40.5982,-124.1573,71.92,74,20,17.27,few clouds
28,28,Sao Jose,BR,-28.2333,-49.1667,71.42,65,100,3.78,overcast clouds
33,33,Ribeira Grande,PT,38.5167,-28.7000,73.38,77,15,4.88,few clouds
40,40,Port Hardy,CA,50.6996,-127.4199,71.78,68,0,10.36,clear sky
54,54,Lagoa,PT,39.0500,-27.9833,72.21,75,2,3.51,clear sky
72,72,Kishi,NG,9.0833,3.8500,70.70,96,100,5.53,moderate rain
76,76,Riachao Das Neves,BR,-11.7461,-44.9100,74.52,35,1,3.06,clear sky
78,78,Lagos,NG,6.5833,3.7500,74.75,87,100,6.35,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 240 entries, 5 to 1863
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   City_ID      240 non-null    int64  
 1   City         240 non-null    object 
 2   Country      240 non-null    object 
 3   Lat          240 non-null    float64
 4   Lng          240 non-null    float64
 5   Max Temp     240 non-null    float64
 6   Humidity     240 non-null    int64  
 7   Cloudiness   240 non-null    int64  
 8   Wind Speed   240 non-null    float64
 9   Description  240 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 20.6+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df

clean_df = clean_df.rename(columns = {"Description":"Current Description"})


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,73.47,broken clouds,-23.1203,-134.9692,
15,Tura,IN,73.72,scattered clouds,25.5198,90.2201,
21,Fortuna,US,71.92,few clouds,40.5982,-124.1573,
28,Sao Jose,BR,71.42,overcast clouds,-28.2333,-49.1667,
33,Ribeira Grande,PT,73.38,few clouds,38.5167,-28.7000,
40,Port Hardy,CA,71.78,clear sky,50.6996,-127.4199,
54,Lagoa,PT,72.21,clear sky,39.0500,-27.9833,
72,Kishi,NG,70.70,moderate rain,9.0833,3.8500,
76,Riachao Das Neves,BR,74.52,clear sky,-11.7461,-44.9100,
78,Lagos,NG,74.75,overcast clouds,6.5833,3.7500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
   
     # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])

clean_hotel_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 240 entries, 5 to 1863
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 240 non-null    object 
 1   Country              240 non-null    object 
 2   Max Temp             240 non-null    float64
 3   Current Description  240 non-null    object 
 4   Lat                  240 non-null    float64
 5   Lng                  240 non-null    float64
 6   Hotel Name           240 non-null    object 
dtypes: float64(3), object(4)
memory usage: 15.0+ KB


In [10]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))